In [1]:
#libraries
import numpy as np
import os
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from scipy.signal import find_peaks #add to documentation
#import pandas as pd
import re
import csv
import sys #check
import stat #check

import tkinter as tk
from tkinter import *
from tkinter import ttk
from tkinter import filedialog
from tkinter.messagebox import showinfo
from tkinter import filedialog as fd

from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.figure import Figure

matplotlib.use('TkAgg')

In [2]:
#functions#

#searching file to find number of measurements
def list_of_measurements():
    #pattern for searching first and list line of measurement in the file
    pattern_all = r'#S\s\d+'
    pattern_rscan = r'#S\s\d+\srscan'

    total = 0
    #searching data from specified measurement
    for L in Lines:
        if re.search(pattern_all, L):
            total += 1
    
    rscans.clear()
    #only numbers of rscan
    for L in Lines:
        search_res = re.search(pattern_rscan, L)
        if search_res:
            text = search_res.group()
            number = re.search(r'\d+',text)
            rscans.append(int(number.group()))
    
    return total
            
#interaction with plot (get points from the plot)
def clicking(event):
    global x_man, y_man
    
    if event.button == 1: #picking a point
        closest_val = min(enumerate(data_x), key=lambda x: abs(x[1] - event.xdata)) #closest x value in plot to clicked point
        cl_val_index = closest_val[0]
        x_man.append(data_x[cl_val_index]) #tabs with choosed peaks
        y_man.append(norm_data_y[cl_val_index])
        ax_detect.plot(data_x[cl_val_index], norm_data_y[cl_val_index], '*', color = 'green', label = 'Choosen peak')
        canvas_detect.draw()
        
        act_xposition.set(str(data_x[cl_val_index])) #display coordiantes of last selected point
        act_yposition.set(str(norm_data_y[cl_val_index]))
    elif event.button == 3: #removing last point
        if not no_peaks: #2 peaks were found in automatic mode
            a_peaks_x = [peak_x[0],peak_x[-1]] #automatically found peaks
            a_peaks_y = [peak_y[0],peak_y[-1]]
        
        ax_detect.clear()
        plotter(actual_nb,ax_detect,'sub and normalized', actual_w, actual_iter_nb) #plotting spectra
        if not no_peaks:
            ax_detect.plot(a_peaks_x, a_peaks_y, '*', color = 'orange', label ='Automatic-searched peaks')
        ax_detect.set_xlabel('Energy (eV)')
        ax_detect.set_ylabel('Intensity (a.u.)')
        ax_detect.set_title('Meusurement number {}'.format(actual_nb))
        ax_detect.legend(handles=[legend_auto, legend_man, legend_spectra])
        #remove one point from choosen peaks tab
        x_man.pop()
        y_man.pop()
        
        ax_detect.plot(x_man, y_man,'*', color = 'green', label = 'Choosen peaks') #plotting the remaining points
        canvas_detect.draw()

#normalization of points to 0 - 1
def normalization(val, mini, maxi):
    return (val-mini)/(maxi-mini)

#normalization of whole vector
def normalization_vec(data):
    max_i = max(data)
    min_i = min(data)
    norm_data_y = [normalization(val, min_i, max_i) for val in data]
    return norm_data_y

#RL3 for automatic mode
def RL3_auto(peaks):
    return peaks[0]/peaks[-1]

#RL3 for manual mode
def RL3_man(peak1, peak2):
    return peak1/peak2

#delta RL3 coefficient 
def delta_RL3(RL3_0, RL3_60):
    dRL3 = []
    for i in range(len(RL3_0)):
        dRL3.append(RL3_0[i]-RL3_60[i])
    return dRL3

#uploading specified measurement
def search_in_file(nb):
        #pattern for searching first and list line of measurement in the file
    nb_lines_comment = 28 #28 is number of comment lines
    pattern_first = r'#S\s{}'.format(nb) #start of data
    pattern_last = r'#C\sAcquisition\sended' #end of data
    
    #searching data from specified measurement
    first_line = 0
    for L in Lines: #searching for the beginning of data in uploaded file
        if re.search(pattern_first, L):
            break
        first_line +=1
    data_start = first_line + nb_lines_comment 
    if data_start - nb_lines_comment == len(Lines): #there is not specified measurement in the file
        return None
    
    last_line = first_line
    for line_nb in range(first_line, len(Lines)): #searching for the end of data in uploaded file
        if re.search(pattern_last, Lines[line_nb]):
            break
        last_line +=1
    return (data_start, last_line) #return tuple with first and last line of measurement data

#choosing points on a plot in the manual mode
def picking_points(nb, w = 0, iter_nb = 0):
   #automatic searched-peaks
    no_peaks = False #variable for automatic mode exception (false - no exception)
    try:  #2 peaks were found in automatic mode
        a_peaks_x = [peak_x[0],peak_x[-1]]
        a_peaks_y = [peak_y[0],peak_y[-1]]
    except:
        no_peaks = True
    
    global actual_nb, actual_w , actual_iter_nb #nb of actual displayed spectra (essential for "clicking" function)
    actual_nb = nb
    actual_w = w
    actual_iter_nb = iter_nb
    
    plotter(nb,ax_detect,'sub and normalized', w, iter_nb) #plotting spectra
    #plotting automatic searched-peaks
    if not no_peaks:
        ax_detect.plot(a_peaks_x, a_peaks_y, '*', color = 'orange', label='Automatic-searched peaks')
    ax_detect.set_xlabel('Energy (eV)')
    ax_detect.set_ylabel('Intensity (a.u.)')
    ax_detect.set_title('Meusurement number {}'.format(nb))
    ax_detect.legend(handles=[legend_auto, legend_man, legend_spectra])
    
    fig_detect.canvas.mpl_connect("button_press_event", clicking) #mark points on the graph
    
    canvas_detect.draw()

#plotting specifed measurement
def plotter(measurement_nb, ax, mode, w = 0, iter_nb = 0):
    global plot_obj
    line_range = search_in_file(measurement_nb) #tuple with first and last line of measurement data

    #convert data into two columns
    filew = open('temp.txt', 'w')
    filew.writelines(Lines[line_range[0]:line_range[1]])
    filew.close()
    data = np.genfromtxt('temp.txt', usecols = (1,3))

    #split imported data to X and Y
    data_x.clear()
    data_y.clear()
    
    #Checking if measurement is properly imported (2 columns) and not empty
    if len(data) > 0 and type(data[0]) == type(np.array(0)): 
        #Checking if measurement is properly imported (2 columns) and not empty
        for point in data:
            data_x.append(point[0])
            data_y.append(point[1])

    channel_nb = line_range[1] - line_range[0]
    measurement_nb = int(measurement_nb)
    w = int(w)
    iter_nb = int(iter_nb)
    
    if mode == 'virgin':
        if w and iter_nb:
            plot_obj = ax.plot(data_x, background(channel_nb, w, iter_nb))
        else:
            plot_obj = ax.plot(data_x, data_y, label = '{}'.format(measurement_nb))
    elif mode == 'sub and normalized':
        if w and iter_nb: #background is removed
            result = list(map(sub, data_y, background(channel_nb, w, iter_nb)))
            norm_data_y = normalization_vec(result)
        else: #no background removal
            norm_data_y = normalization_vec(data_y)
        plot_obj = ax.plot(data_x, norm_data_y, label = '{}'.format(measurement_nb))

#ddetecting peaks on the plots (automatic and manual mode)
def peaks_detection(nb, w = 0, iter_nb = 0):
    #Load data and calculation for SROT 0
    nb = int(nb)
    w = int(w)
    iter_nb = int(iter_nb)
    line_range = search_in_file(nb) #tuple with first and last line of measurement data
           
    #convert data into two columns
    filew = open('temp.txt', 'w')
    filew.writelines(Lines[line_range[0]:line_range[1]])
    filew.close()
    data = np.genfromtxt('temp.txt', usecols = (1,3))

    #split imported data to X and Y
    global data_x, data_y, no_peaks
    data_x.clear()
    data_y.clear()
    no_peaks = False  #variable for automatic mode exception (false - no exception)
    for point in data:
        data_x.append(point[0])
        data_y.append(point[1])

    if w and iter_nb: #background is removed
        channel_nb = line_range[1] - line_range[0]
        data_y = list(map(sub, data_y, background(channel_nb, w, iter_nb)))

    #normalization
    global norm_data_y
    norm_data_y = normalization_vec(data_y)

    #finding peaks
    peak, _ = find_peaks(norm_data_y, height = 0.3)

    #preparing peaks for plotting 
    peak_x.clear()
    peak_y.clear()
    for peak_nb in peak:
        peak_x.append(data_x[peak_nb])
        peak_y.append(norm_data_y[peak_nb])
    
    if mode.get() == 'automatic':
        try:
            RL3_tab.append(RL3_auto(peak_y)) #adding auto-searched peaks to results
        except:
            no_peaks = True #less than 2 peaks found in automatic mode
    if mode.get() == 'manual' or no_peaks:
        x_man.clear()
        y_man.clear()

        picking_points(int(nb), w, iter_nb)

#accepting automatically searched peaks or adding choosen peaks to results  
def pick_manual(prev_nb):
    global man_nb
    if len(y_man) == 2: #auto-searched peaks denied
        RL3_tab.append(RL3_man(y_man[0],y_man[1])) #adding manual choosen peaks to results
        man_peaks[prev_nb] = ((x_man[0],y_man[0]), (x_man[1],y_man[1]))
    elif len(y_man) == 0: #auto-searched peaks accepted and 2 peaks found
        try:
            RL3_tab.append(RL3_auto(peak_y)) #adding auto-searched peaks to results
            man_peaks[prev_nb] = ((peak_x[0],peak_y[0]), (peak_x[-1],peak_y[-1]))
        except: ##less than 2 peaks found in automatic mode
            man_nb -= 1
            popup_nopeaks()

    else: #repeat detection for last measurement
        man_nb -= 1
        popup_manmode() #popup window informing about mistake in choosing peaks

#clear all elements in tree
def clear_all_tree(tree):
    for item in tree.get_children():
        tree.delete(item)

In [3]:
#####Uploading tab

#upload_btn - open file explorer to upload file
def select_file():
    global file_path, Lines, total
    filetypes = (('dat files', '*.dat'), ('All files', '*.*'))
    file_path = fd.askopenfilename(title='Open a file', initialdir='/', filetypes=filetypes)
    
    #import file line by line
    data_set = open(file_path, 'r')
    Lines = data_set.readlines() #all lines from file
    data_set.close()
    total = list_of_measurements()
    ms_list.delete(0, END) #clearing listbox with measurements
    for nb in range(1,total+1): #filling listbox with measurements numbers
        ms_list.insert(END, str(nb))
    check_rscan.grid(row = 4, column = 0, sticky = W)
    
#Adding and removing measurements for calculations
#For srot 0
def choice0():
    inlist = selected0.get(0,END) #already selected elements
     
    for i in ms_list.curselection():
        m_nb = ms_list.get(i,i)[0]
        if m_nb not in inlist:
            selected0.insert(END, m_nb)
    for i in selected0.curselection():
        selected0.delete(i)
    only_rscan()

#For srot 60
def choice60():
    inlist = selected60.get(0,END) #already selected elements
    
    for i in ms_list.curselection():
        m_nb = ms_list.get(i,i)[0]
        if m_nb not in inlist:
            selected60.insert(END, m_nb)
    for i in selected60.curselection():
        selected60.delete(i)
    only_rscan()

#Plotting marked uploaded spectras
def plotting_up(event):
    ax_up.clear()
    for nb in ms_list.curselection(): #iterating through measurements list
        m_nb = ms_list.get(nb,nb)[0]
        plotter(int(m_nb),ax_up,'virgin')
      
    ax_up.legend()
    canvas_up.draw()

#Clear_plot button - clearing all plots
def clear():
    ax_up.clear()
    canvas_up.draw()
    only_rscan()

#check_rscan - showing only rscans in measurement list
def only_rscan():
    ms_list.delete(0, END)
    if if_only.get() == 0:
        for nb in range(1,total+1):
            ms_list.insert(END, str(nb))
    else:
        for nb in rscans:
            ms_list.insert(END, str(nb))

#new_session_btn - clear all tabs and listboxes - preparing for new data set for calculations
def new_session():
    #Disabling tabs
    notebook.tab(back_rem, state = 'disabled')
    notebook.tab(calculations, state = 'disabled')
    notebook.tab(detection, state = 'disabled')
    notebook.tab(results_tab, state = 'disabled')
    
    #Clearing all tabs with results
    accepted_all.clear()
    accepted.clear()
    accepted_p.clear()
    RL3_tab.clear()
    dRL3_tab.clear()
    srot0el.clear()
    srot60el.clear()
    #Clear listboxes
    selected0.delete(0, END)
    selected60.delete(0, END)
    back_list.delete(0, END)
    clear_all_tree(res0_tree)
    clear_all_tree(res60_tree)
    clear_all_tree(resdRL3_tree)
    #Clear axis
    ax_back.clear()
    ax_calc.clear()
    ax_detect.clear()
    #Set default values foe checkboxes
    check_noback.deselect()
    check_oneback.deselect()
    #Set normal state to all widgets in background tab
    undo_btn["state"] = "normal"
    acc_btn["state"] = "normal"
    sub_btn["state"] = "normal"
    w_widget["state"] = "normal"
    iter_widget["state"] = "normal"
    check_oneback["state"] = "normal"

def popup_nochoice(): #popup window when there aren't any measurements selected
    popup_upload = Toplevel(window)
    popup_upload.geometry("550x200")
    Label(popup_upload, text= "Choose measurements for calculations!",
          font= ('Helvetica 18 bold')).place(relx=.5, rely=.5,anchor= CENTER)    

#accept saving choosen measurements numbers
def accept():
    global measurements0
    global measurements60
    measurements0 = selected0.get(0,END) #lists with choosen measurements (srot 0 and srot 60)
    measurements60 = selected60.get(0,END)
    if measurements0 == () and measurements60 == (): #no choosen measurements
        popup_nochoice()
    else: #inserting measurements to list displayed in background tab
        for nb in measurements0:
            back_list.insert(END, nb)
        for nb in measurements60:
            back_list.insert(END, nb)
        notebook.tab(back_rem, state = 'normal')
        notebook.select(1) #switch to background tab
    
#####background removal functions

#calculating background fit
def background(channel, w, iter_nb):
    y_mean = data_y.copy()
    temp = data_y.copy()
    for nb in range(iter_nb):
        for i in range(w, channel-w):
            mean = (temp[i-w] + temp[i+w])/2
            if y_mean[i] > mean:
                temp[i] = mean
        y_mean = temp.copy()
    return y_mean #background calculated based on w and number of iterations

#w_widget and iter_widget spinboxes - changing value of fitting parameters
def change_w():
    global w
    w = int(w_widget.get())
    plotting_back()
    
def change_iter():
    global iter_nb
    iter_nb = int(iter_widget.get())
    plotting_back()

#sub_btn - subtraction of two spectra
def clicked_sub():
    ax_back.clear()
    nb = back_list.curselection()  
    m_nb = back_list.get(nb,nb)[0] #getting number of highlighted measurement
    plotter(int(m_nb), ax_back,'sub and normalized', w, iter_nb) #plotting spectra after removing background
    canvas_back.draw()
    
    #packing widgets
    undo_btn.grid(row = 3, column = 1)
    acc_btn.grid(row = 4, column = 1)
    sub_btn.grid_forget()
    w_widget["state"] = "disable"
    iter_widget["state"] = "disable"

#undo_btn - rejection of subtraction and back to background fitting
def clicked_undo():
    plotting_back()
    
    #packing widgets
    sub_btn.grid(row = 3, column = 1, rowspan = 2)
    undo_btn.grid_forget()
    acc_btn.grid_forget()
    w_widget["state"] = "normal"
    iter_widget["state"] = "normal"

#subtraction
def sub(a,b):
    return a-b

#plotting spectrum with background fit
def plotting_back():
    ax_back.clear()
    nb = back_list.curselection()
    m_nb = back_list.get(nb,nb)[0]
    plotter(int(m_nb), ax_back,'virgin') #plotting spectra (unnormalized)
    plotter(int(m_nb), ax_back,'virgin', w, iter_nb) #plotting background for actual parameters
    canvas_back.draw()

#mark measurement in list (plotting and buttons managing)
def plotting_marked(event):
    nb = back_list.curselection()
    m_nb = back_list.get(nb,nb)[0]
    if m_nb not in accepted: #background wasn't removed from choosen spectrum
        clicked_undo()
    else: #background was removed from choosen spectrum earlier
        clicked_sub()
        undo_btn.grid_forget()
        acc_btn.grid_forget()

#acc_btn - accepting parameters of background fit        
def acc_back():
    nb = back_list.curselection()
    m_nb = back_list.get(nb,nb)[0] #getting number of highlighted measurement
    accepted.append(m_nb) #adding measurement to the list of already accepted spectra
    if m_nb in measurements0:
        #dictionary with parameters (nb of measurement - key, (w, nb of iterations, srot) - element)
        accepted_p[m_nb] =  (w_widget.get(),iter_widget.get(), 0) #adding nb and parameters of backround to dictionary
    elif m_nb in measurements60:
        accepted_p[m_nb] = (w_widget.get(),iter_widget.get(), 60)
    undo_btn.grid_forget()
    acc_btn.grid_forget()

#Popup window when not all measurements have background set
def popup_background(): #popup window when there aren't selected any background removal settings
    popup_back = Toplevel(back_rem)
    popup_back.geometry("550x200")
    Label(popup_back, text= "Choose background settings \nfor all measurements!",
          font= ('Helvetica 18 bold')).place(relx=.5, rely=.5,anchor= CENTER)

#accall_btn - accepting background settings, go to calculations tab and plot all spectra (normalized)
def acc_all():
    w = w_widget.get()
    iter_nb = iter_widget.get()
    if if_noback.get() == 1 or if_oneback.get() == 1: #no background removal
        for nb in accepted[0]: #plotting normalized srot 0 measurements
            plotter(nb, ax_calc, 'sub and normalized', w, iter_nb)
        for nb in accepted[1]: #plotting normalized srot 60 measurements
            plotter(nb, ax_calc, 'sub and normalized', w, iter_nb)
    else: #subtract the unique backgroung from every measurement
        for el in accepted_p:
            nb = el
            w = accepted_p[el][0]
            iter_nb = accepted_p[el][1]
            plotter(nb, ax_calc, 'sub and normalized', w, iter_nb) #plotting normalized spectra after bckg removing
    ax_calc.set_xlabel('Energy (ev)')
    ax_calc.set_ylabel('Intensity')
    ax_calc.legend()
    canvas_calc.draw()
    if if_noback.get() == 1 or if_oneback.get() == 1 or len(accepted_p) == back_list.size():
        notebook.tab(calculations, state = 'normal')
        notebook.select(2) #switch to calculations tab
    else: #there is no choosen option of bckg removing for all measurements
        popup_background()
    
#check_noback - enabling and disabling of background fitting option (button managing)
def noback():
    if if_noback.get() == 0:
        w_init.set(w)
        iter_init.set(iter_nb)
        accepted.clear()
        #enabling all widgets
        undo_btn["state"] = "normal"
        acc_btn["state"] = "normal"
        sub_btn["state"] = "normal"
        w_widget["state"] = "normal"
        iter_widget["state"] = "normal"
        check_oneback["state"] = "normal"
    else:
        w_init.set("0")
        iter_init.set("0")
        accepted.append(measurements0)
        accepted.append(measurements60)
        for nb in measurements0: #saving parameters of background
            accepted_p[nb] = (0,0,0) #(w, iter_nb, srot)
        for nb in measurements60:
            accepted_p[nb] = (0,0,60)
        #disabling all widgets despite of noback checkbox
        undo_btn["state"] = "disable"
        acc_btn["state"] = "disable"
        sub_btn["state"] = "disable"
        w_widget["state"] = "disable"
        iter_widget["state"] = "disable"
        check_oneback["state"] = "disable"

#check_oneback - enabling and disabling of option of changing parameters
def oneback():
    if if_oneback.get() == 0:
        #enabling all widgets
        w_widget["state"] = "normal"
        iter_widget["state"] = "normal"
        acc_btn["state"] = "normal"
        check_noback["state"] = "normal"
    else:
        accepted.append(measurements0)
        accepted.append(measurements60)
        for nb in measurements0: #saving parameters of background
            accepted_p[nb] = (w_widget.get(), iter_widget.get(), 0) #(w, iter_nb, srot)
        for nb in measurements60:
            accepted_p[nb] = (w_widget.get(), iter_widget.get(), 60)
        #disabling all widgets despite of oneback checkbox, sub_btn and undo_btn
        w_widget["state"] = "disable"
        iter_widget["state"] = "disable"
        acc_btn["state"] = "disable"
        check_noback["state"] = "disable"
        
#####Calculations window
#Popup window - less than 2 peaks found in automatic mode
def popup_nopeaks():
    popup_peaks = Toplevel(detection)
    popup_peaks.geometry("550x200")
    Label(popup_peaks, text= "Peaks searching failed.\nYou have to choose peaks manually.",
          font= ('Helvetica 18 bold')).place(relx=.5, rely=.5,anchor= CENTER) 

#detect_btn - go to detection tab and draw first spectrum if manual mode is on
def detect():
    repeat_btn.pack_forget()
    global man_nb #counting processed measurements
    w = w_widget.get()
    iter_nb = iter_widget.get()
    keys_list = list(accepted_p.keys())
    ax_detect.clear()
    RL3_tab.clear()
    
    if mode.get() == 'manual': #manual mode
        position_fr.grid(row = 0, column = 0) #widget for displaying coordinates of last point
        acc_man_btn.grid(row = 0, column = 0, rowspan = 2) #button for accepting manual choosen peaks
        go_results_btn.grid_forget()
        repeat_btn.grid_forget()
        man_nb = 1
        nb = keys_list[0]
        w = accepted_p[nb][0]
        iter_nb = accepted_p[nb][1]
        #Load data and calculation for SROT 0
        peaks_detection(nb, w, iter_nb)
    elif mode.get() == 'automatic': #automatic mode
        acc_man_btn.grid_forget()
        position_fr.grid_forget()
        go_results_btn.grid(row = 0, column = 0, sticky = S, padx = 10, pady = 10)
        repeat_btn.grid(row = 1, column = 0, sticky = N, padx = 10, pady = 10)
        
        try:
            for el in accepted_p: #plotting all spectra with marked automatically searched peaks
                nb = el
                w = accepted_p[el][0]
                iter_nb = accepted_p[el][1]
                peaks_detection(nb, w, iter_nb) 
                plt_color = plot_obj[-1].get_color() #color of last plot
                ax_detect.plot([peak_x[0], peak_x[-1]], [peak_y[0], peak_y[-1]],'*', color = plt_color)
                plotter(nb, ax_detect, 'sub and normalized', w, iter_nb)
                ax_detect.legend()
        except: #auto-searching failed
            popup_nopeaks()
            mode_man.invoke()
            detect()
            
    notebook.tab(detection, state = 'normal')
    notebook.select(3) #switch to detection tab


#####Detection tab
#popup window in manual mode - not exactly two peaks was picked
def popup_manmode():
    popup_detect = Toplevel(detection)
    popup_detect.geometry("550x200")
    if len(y_man) == 1:
        Label(popup_detect,
              text= "You have to choose two peaks or\naccept automatically searched peaks (click Accept)",
              font= ('Helvetica 18 bold')).place(relx=.5, rely=.5,anchor= CENTER)
    if len(y_man) > 2:
         Label(popup_detect,
               text= "You pick more than two peaks. Automated-searched peaks will be used for calculations",
               font= ('Helvetica 18 bold')).place(relx=.5, rely=.5,anchor= CENTER)
            
#acc_peaks_btn - accept picked peaks in manual mode and plot next spectra
def acc_peaks():
    global man_nb
    keys_list = list(accepted_p.keys())
    #CHECK!
    #if len(y_man) == 1 or len(y_man) > 2: #popup window informing about mistake in choosing peaks
        popup_manmode()
        
    prev_nb = keys_list[man_nb-1] 
    pick_manual(prev_nb) #saving calculated parameters
    
    ax_detect.clear()
    ax_detect.set_xlabel('Energy (ev)')
    ax_detect.set_ylabel('Intensity')

    if man_nb < len(keys_list): #not all spectra have been displayed yet
        nb = keys_list[man_nb]
        w = accepted_p[nb][0]
        iter_nb = accepted_p[nb][1]
        peaks_detection(nb, w, iter_nb) #pick peaks or accept automatic-searched peaks
            
        man_nb += 1
    else: #all spectra are analyzed
        for el in accepted_p: #plotting all spectra
            nb = el
            w = accepted_p[el][0]
            iter_nb = accepted_p[el][1]
            plotter(nb, ax_detect, 'sub and normalized', w, iter_nb)
            plt_color = plot_obj[-1].get_color() #color of last plot
            ax_detect.plot([man_peaks[nb][0][0], man_peaks[nb][1][0]],
                           [man_peaks[nb][0][1], man_peaks[nb][1][1]],'*', color = plt_color)
            ax_detect.legend()
        #showing and hiding buttons buttons
        repeat_btn.grid(row = 1, column = 0, sticky = N, padx = 10, pady = 10)
        go_results_btn.grid(row = 0, column = 0, sticky = S, padx = 10, pady = 10)
        acc_man_btn.grid_forget()
        position_fr.grid_forget()

        ax_detect.legend()
        canvas_detect.draw()
#repeat_btn - set manual mode and start analysis of peaks
def repeat_manual():
    mode.set('manual')
    detect()

#Popup window - not all measurements was processed
def popup_results():
    popup_res = Toplevel(detection)
    popup_res.geometry("550x200")
    Label(popup_res,
          text= "First you have to go\nthrough all selected measurements",
          font= ('Helvetica 18 bold')).place(relx=.5, rely=.5,anchor= CENTER)    
    
#Go to results tab and display RL3 and delta RL3 values
def display_results():
    keys_list = list(accepted_p.keys())
    if mode.get() == 'manual' and man_nb < len(keys_list): #not all measurements was processed
        popup_results()
    else:
        i = 0 #index of RL3_tab (subsequent measurements)
        srot0el.clear() #srot 0 measurements (number of measurement, RL3)
        srot60el.clear()
        for el in accepted_p: #calculating RL3 and adding to result trees
            if accepted_p[el][2] == 0:
                res0_tree.insert('', END, values = (el, str(RL3_tab[i])))
                srot0el.append((el, RL3_tab[i]))
            elif accepted_p[el][2] == 60:
                res60_tree.insert('', END, values = (el, str(RL3_tab[i])))
                srot60el.append((el, RL3_tab[i]))
            i += 1

        srot_len = len(srot0el) #number of measurements for delta RL
        if len(srot60el) < srot_len:
            srot_len = len(srot60el)

        for i in range(srot_len): #calculating delta RL and adding to result trees
            dRL3 = srot60el[i][1] - srot0el[i][1]
            dRL3_tab.append(dRL3)
            resdRL3_tree.insert('', END, values = (srot0el[i][0], srot60el[i][0], str(dRL3)))

        #Set labels and name of columns of trees
        label_res0.config(text = '{} for SROT 0'.format(RL_mode.get()))
        label_res60.config(text = '{} for SROT 60'.format(RL_mode.get()))
        label_dRL3.config(text = 'delta {}'.format(RL_mode.get()))
            
        res0_tree.heading('RL factor', text='{}'.format(RL_mode.get()))
        res60_tree.heading('RL factor', text='{}'.format(RL_mode.get()))
        resdRL3_tree.heading('dRL factor', text='delta {}'.format(RL_mode.get()))
            
        notebook.tab(results_tab, state = 'normal')
        notebook.select(4) #go to results tab
    
#Plotting results
def plot_results():
    if if_specify.get() == 1 and len(srot0el) == len(srot60el): #user want to specify x tittle and x values for the plots
        x_tittle = x_tittle_ent.get()
        x_tab = x_val_ent.get().split()
        x_int = list(map(float, x_tab))
        x_int_sr0 = x_int
        x_int_sr60 = x_int
    else: #default values (from 0 to max nb of element)
        x_tittle = 'x'
        x_int = range(len(dRL3_tab))
        x_int_sr0 = range(len(srot0el)) #xvalues from 0 to nb of measurements
        x_int_sr60 = range(len(srot60el))
    
    #create figure
    plt.figure(figsize=(12,9))

    #all spectra
    plt.subplot(2, 2, 1)
    for el in accepted_p:
        m_nb = el
        w = accepted_p[el][0]
        iter_nb = accepted_p[el][1]
        plotter(m_nb, plt.gca(), 'sub and normalized', w, iter_nb)
    plt.title("Spectra", pad = 2.0)
    plt.xlabel("Energy (ev)")
    plt.ylabel("Intensity (a.u.)")
    plt.legend()
    
    RL3_0_tab = [] #list with RL3 factors for srot 0 (for plotting)
    RL3_60_tab = []
        
    for tup in srot0el:
        RL3_0_tab.append(tup[1])
    for tup in srot60el:
        RL3_60_tab.append(tup[1])
        
    #RL for SROT 0
    plt.subplot(2, 2, 2)
    plt.plot(x_int_sr0, RL3_0_tab, marker = 'o')
    plt.title("{} for SROT 0".format(RL_mode.get()), pad = 2.0)
    plt.xlabel("{}".format(x_tittle))
    plt.ylabel("{}".format(RL_mode.get()))

    #RL for SROT 60
    plt.subplot(2, 2, 3)
    plt.plot(x_int_sr60, RL3_60_tab, color = 'green', marker = 'o', label='{} for SROT 60'.format(RL_mode.get()))
    plt.title("{} for SROT 60".format(RL_mode.get()), pad = 2.0)
    plt.xlabel("{}".format(x_tittle))
    plt.ylabel("{}".format(RL_mode.get()))


    #dRL
    plt.subplot(2, 2, 4)
    plt.plot(x_int, dRL3_tab, color = 'red', marker = 'o', label='dRL3')
    plt.title("delta {}".format(RL_mode.get()), pad = 2.0)
    plt.xlabel("{}".format(x_tittle))
    plt.ylabel("d{}".format(RL_mode.get()))

    plt.show()

#check_x_val checkbutton - display textboxes to enter x values for the plots
def specify_x():
    if if_specify.get() == 1:
        x_val_fr.grid(row = 2, column = 3)
    else:
        x_val_fr.grid_forget()
#save_file_btn - saving results to file
def save_to_file():
    files = [('csv file, ','*.csv'), ('All Files', '*.*'), ('Text Document', '*.txt')]
    save_file = fd.asksaveasfile(filetypes = files, defaultextension = files) #open file explorer window to save file
    
    srot_len = len(srot0el) #number of measurements for one srot
    if len(srot60el) > srot_len: #checking for which srot there is more measurements
        srot_len = len(srot60el)
        
    if if_specify.get() == 1: #user want to specify x tittle and x values 
        x_tittle = x_tittle_ent.get()
        x_tab = x_val_ent.get().split()
        x_int = list(map(float, x_tab))
    else:
        x_tittle = 'x'
        x_int = range(srot_len)
    
    with open(save_file.name, 'w+', encoding='UTF8', newline = '') as f: #saving results to csv
        data_csv = csv.writer(f)
        data_csv.writerow([x_tittle, 'Nb of measurement (srot 0)', '{} for SROT 0'.format(RL_mode.get()),
                           'Nb of measurement (srot 0)', '{} for SROT 60'.format(RL_mode.get()),
                           'delta {}'.format(RL_mode.get())])
        for i in range(srot_len):
            if i >= len(srot0el):
                srot0el.append(('',''))
                dRL3_tab.append('')
            elif i >= len(srot60el):
                srot60el.append(('',''))
                dRL3_tab.append('')
            data_csv.writerow([x_int[i], srot0el[i][0], srot0el[i][1], #(value of x, number of measurement srot 0, RL3 srot 0
                               srot60el[i][0], srot60el[i][1], dRL3_tab[i]]) #number of measurement srot 60, RL3 srot 60)
    f.close()

In [4]:
root = Tk()
root.title('XMLD TOOL')

#Creating notebook (window with tabs)
notebook = ttk.Notebook(root)
notebook.pack()

window = Frame(notebook) #uploading tool
back_rem = Frame(notebook) #background removal
calculations = Frame(notebook) #tab with calculations
detection = Frame(notebook) #Peaks detection
results_tab = Frame(notebook) #display results odf calculation

#Packing tabs
window.pack(fill='both', expand=True)
back_rem.pack(fill='both', expand=True)
calculations.pack(fill='both', expand=True)
detection.pack(fill='both', expand=True)
results_tab.pack(fill='both', expand=True)

#Add frames to notebook
notebook.add(window, text='Uploading')
notebook.add(back_rem, text='Background removal')
notebook.add(calculations, text='Calculations')
notebook.add(detection, text='Peaks detection')
notebook.add(results_tab, text = 'Results')

#State of tabs ODKOMENTOWAĆ!
#notebook.tab(window, state = 'normal')
#notebook.tab(back_rem, state = 'disabled')
#notebook.tab(calculations, state = 'disabled')
#notebook.tab(detection, state = 'disabled')
#notebook.tab(results_tab, state = 'disabled')

#####Uploading window
#Preparing for creation of listbox with measurements numbers
file_path = None #path of file with measurements
Lines = None #all lines from imported file
rscans = [] #tab with numbers of rscans
total = None #nb of measurements in file (+ filling rscans)
data_x = [] #lists for imported data
data_y = []

#Creating widgets
#Frames
allms_fr = Frame(window)
srot0_fr = Frame(window)
srot60_fr = Frame(window)

#Tittles for frames
label_all= Label(allms_fr,text="List of measurements")
label_all.pack(side=TOP)
label0= Label(srot0_fr,text="SROT 0 measurements")
label0.pack(side=TOP)
label60= Label(srot60_fr,text="SROT 60 measurements")
label60.pack(side=TOP)

#Scrollbars
scroll_m = Scrollbar(allms_fr, orient = VERTICAL)
scroll_sr0 = Scrollbar(srot0_fr, orient = VERTICAL)
scroll_sr60 = Scrollbar(srot60_fr, orient = VERTICAL)


#Listboxes
ms_list = Listbox(allms_fr, yscrollcommand = scroll_m.set, selectmode = MULTIPLE) #list of all measurements
selected0 = Listbox(srot0_fr, yscrollcommand = scroll_sr0.set, selectmode = MULTIPLE) #list for picked SROT 0 measurements
selected60 = Listbox(srot60_fr, yscrollcommand = scroll_sr60.set, selectmode = MULTIPLE) #list for picked SROT 60 measurements


#Buttons
upload_btn = Button(window, text='Upload file', command=select_file, width=10)
move0 = Button(window, text='Add/Delete', command=choice0) #confirming SROT 0 measurement
move60 = Button(window, text='Add/Delete', command=choice60)
clear_plot = Button(window, text = 'Clear plot', command = clear, width=10)
new_session_btn = Button(window, text = 'New session', command = new_session, width=10) #clear measurements in srot 0 listbox
acc = Button(window, text = 'Accept and go to\nbackground settings', command = accept, width=15) #accept choices and go to background tab

#Checkbuttons
if_only = IntVar()
check_rscan = Checkbutton(window, text='Show only rscans', variable=if_only, command = only_rscan)

#Packing widgets
#Frames
allms_fr.grid(row = 0, column = 0, rowspan = 4, padx = 10, pady = 10)
srot0_fr.grid(row = 5, column = 0)
srot60_fr.grid(row = 5, column = 1)

#Listboxes
ms_list.pack(side = LEFT, fill = BOTH)
selected0.pack(side = LEFT, fill = BOTH)
selected60.pack(side = LEFT, fill = BOTH)

#Buttons
upload_btn.grid(row = 1, column = 1, pady = 5, sticky = N)
move0.grid(row = 6, column = 0)
move60.grid(row = 6, column = 1)
clear_plot.grid(row = 2, column = 1, pady = 5, sticky = S)
new_session_btn.grid(row = 0, column = 1, pady = 5, sticky = S)
acc.grid(row = 3, column = 1, pady = 5, sticky = N)

#Scrollbars
scroll_m.pack(side = RIGHT, fill = Y)
scroll_m.config(command = ms_list.yview)
scroll_sr0.pack(side = RIGHT, fill = Y)
scroll_sr0.config(command = selected0.yview)
scroll_sr60.pack(side = RIGHT, fill = Y)
scroll_sr60.config(command = selected60.yview)

#add figure to window
fig_up = Figure(figsize=(7, 5), dpi=100)
ax_up = fig_up.add_subplot(111)
# Create a FigureCanvasTkAgg object to display the figure in the Tkinter window
canvas_up = FigureCanvasTkAgg(fig_up, master=window)
canvas_up.draw()

# Pack the canvas widget to display it in the window
canvas_up.get_tk_widget().grid(row = 0, column = 2, rowspan = 6, padx = 10)

ms_list.bind('<<ListboxSelect>>', plotting_up) #plotting marked spectra

#####Background removal

#Parameters for background fitting
w = 4
iter_nb = 50

#Initial values for widgets
w_init = StringVar(back_rem)
iter_init = StringVar(back_rem)
w_init.set(str(w))
iter_init.set(str(iter_nb))

accepted_all = [] #numbers of measurments after background removal
accepted = [] #numbers of measurments after background removal (split into srot 0 and srot 60)
accepted_p = {} #background parameters of measurments

#Creating widgets
#Frames
back_fr = Frame(back_rem)
parameters_fr = Frame(back_rem)
check_fr = Frame(back_rem)

#Labels (tittles)
label_back= Label(back_fr,text="List of measurements")
label_w = Label(parameters_fr, text="Width")
label_itr = Label(parameters_fr, text="Iterations")
label_parameters = Label(parameters_fr, text="Parameters")

#Listboxes
back_list = Listbox(back_fr, yscrollcommand = scroll_m.set) #list of all measurements

#Buttons
sub_btn = Button(back_rem, text = 'Subtraction' , command = clicked_sub)
undo_btn = Button(back_rem, text = 'Undo', command = clicked_undo)
acc_btn = Button(back_rem, text = 'Accept', command = acc_back)
accall_btn = Button(back_rem, text = 'Accept all and\ngo to calculations', command = acc_all, height = 3, width = 18)

#Checkbuttones
if_noback = IntVar()
check_noback = Checkbutton(check_fr, text = "Don't subtract the background", variable=if_noback, command = noback)
if_oneback = IntVar()
check_oneback = Checkbutton(check_fr, text = "Use actual parameters to all",
                            variable=if_oneback, command = oneback)

#Spinboxes
w_widget = Spinbox(parameters_fr, from_ = 0, to = 10, command = change_w, textvariable = w_init)
iter_widget = Spinbox(parameters_fr, from_ = 0, to = 100, command = change_iter, textvariable = iter_init)

#Scrollbars
scroll_back = Scrollbar(back_fr, orient = VERTICAL)

#Packing widgets
#Frames
back_fr.grid(row = 0, column = 0)
check_fr.grid(row = 1, column = 0, sticky = N)
parameters_fr.grid(row = 2, column = 0)

#Labels (tittles)
label_back.pack(side=TOP)
label_w.grid(row = 1, column = 0)
label_itr.grid(row = 2, column = 0)
label_parameters.grid(row = 0, column = 0, columnspan = 2)

#Listboxes
back_list.pack(side = LEFT, fill = BOTH)

#Buttons
sub_btn.grid(row = 3, column = 1, rowspan = 2)
accall_btn.grid(row = 3, column = 0, rowspan = 2, padx = 10, pady = 10)

#Checkbuttons
check_noback.grid(row = 0, column = 0, sticky = W)
check_oneback.grid(row = 1, column = 0, sticky = W)

#Spinboxes
w_widget.grid(row = 1, column = 1)
iter_widget.grid(row = 2, column = 1)

#Scrollbars
scroll_back.pack(side = RIGHT, fill = Y)
scroll_back.config(command = ms_list.yview)

# Create figure and plot with spectrum
fig_back = Figure(figsize=(7, 5), dpi=100)
ax_back = fig_back.add_subplot(111)

# Create a FigureCanvasTkAgg object to display the figure in the Tkinter window
canvas_back = FigureCanvasTkAgg(fig_back, master=back_rem)
canvas_back.draw()

# Pack the canvas widget to display it in the window
canvas_back.get_tk_widget().grid(row = 0, column = 1, rowspan = 3)

back_list.bind('<<ListboxSelect>>', plotting_marked) #plotting spectra

#####Calculation tab
plot_obj = None #store ax.plot object (getting colors of plots)
#Lists for RL3 and delta RL3 results
RL3_tab = []
dRL3_tab = []

#Lists for automatic-searched peaks
peak_x = []
peak_y = []

#Lists for manual choosen peaks
x_man = []
y_man = []
man_peaks = {}

#legend for plotting in manual mode
legend_auto = mpatches.Patch(color='orange', label='Automatic-searched peaks')
legend_man = mpatches.Patch(color='green', label='Manual choosen peaks')
legend_spectra = mpatches.Patch(color='blue', label='Spectra')

#Creating widgets
#Frames
detect_mode_fr = Frame(calculations)
calc_mode_fr = Frame(calculations)

#Labels
label_dmode = Label(detect_mode_fr, text = "Detection mode")
label_cmode = Label(calc_mode_fr, text = "Calculation mode")
#Buttons
detect_btn = Button(calculations, text = 'Detect peaks' , command = detect, height = 3, width = 18)

#Radiobuttons
mode = StringVar(value="automatic")
mode_man = Radiobutton(detect_mode_fr, text='Manual', variable=mode, value='manual')
mode_aut = Radiobutton(detect_mode_fr, text='Automatic', variable=mode, value='automatic')

RL_mode = StringVar(value="RL3")
mode_RL2 = Radiobutton(calc_mode_fr, text='RL2', variable=RL_mode, value='RL2')
mode_RL3 = Radiobutton(calc_mode_fr, text='RL3', variable=RL_mode, value='RL3')

#Packing widgets
#Frames
detect_mode_fr.grid(row = 0, column = 0, sticky = N, padx = 10, pady = 10)
calc_mode_fr.grid(row = 0, column = 1, sticky = N, padx = 10, pady = 10)

#Labels
label_dmode.pack()
label_cmode.pack()

#Buttons
detect_btn.grid(row = 1, column = 0, columnspan = 2, sticky = N)

#Radiobuttons
mode_man.pack(anchor=W)
mode_aut.pack(anchor=W)
mode_RL2.pack()
mode_RL3.pack()

# Create figure and plot with spectrum
fig_calc = Figure(figsize=(8, 5.5), dpi=100)
ax_calc = fig_calc.add_subplot(111)
ax_calc.set_xlabel('Energy (ev)')
ax_calc.set_ylabel('Intensity')

# Create a FigureCanvasTkAgg object to display the figure in the Tkinter window
canvas_calc = FigureCanvasTkAgg(fig_calc, master=calculations)
canvas_calc.draw()

# Pack the canvas widget to display it in the window
canvas_calc.get_tk_widget().grid(row = 0, column = 2, rowspan = 2)

#####Detection window
man_nb = None #number of actual displayed spectra in manual mode (next spectra += 1)

#Create widgets
#Frames
position_fr = Frame(detection)

#Buttons
acc_man_btn = Button(detection, text = 'Accept peaks' , command = acc_peaks)
go_results_btn = Button(detection, text = 'Go to results' , command = display_results)
repeat_btn = Button(detection, text = 'Repeat analysis in manual mode' , command = repeat_manual)

#Entries
act_xposition = StringVar()
act_xposition.set("")
act_yposition = StringVar()
act_yposition.set("")

ent_xposttion = Entry(position_fr, textvariable=act_xposition, state="readonly")
ent_yposttion = Entry(position_fr, textvariable=act_yposition, state="readonly")

#Labels (tittles for entries)
label_xposition = Label(position_fr, text="X value")
label_yposition = Label(position_fr, text="Y value")
label_coordinates = Label(position_fr, text="Last selected point")

#Packing widgets
#Frames
position_fr.grid(row = 0, column = 0)

#Labels
label_xposition.grid(row = 1, column = 0)
label_yposition.grid(row = 2, column = 0)
label_coordinates.grid(row = 0, column = 0, columnspan = 2)

#Buttons
acc_man_btn.grid(row = 0, column = 0, sticky = S, padx = 10, pady = 10)
go_results_btn.grid(row = 0, column = 0, sticky = S, padx = 10, pady = 10)
repeat_btn.grid(row = 1, column = 0, sticky = N, padx = 10, pady = 10)

#Entries
ent_xposttion.grid(row = 1, column = 1)
ent_yposttion.grid(row = 2, column = 1)

# Create figure and plot with spectrum
fig_detect = Figure(figsize=(8, 5.5), dpi=100)
ax_detect = fig_detect.add_subplot(111)

# Create a FigureCanvasTkAgg object to display the figure in the Tkinter window
canvas_detect = FigureCanvasTkAgg(fig_detect, master=detection)
canvas_detect.draw()

# Pack the canvas widget to display it in the window
canvas_detect.get_tk_widget().grid(row = 0, column = 1, rowspan = 2)

#####Results window
#Lists for tuples - (measurement nb, RL3)
srot0el = [] 
srot60el = []

#Creating widgets
#Frames
srot0_results = Frame(results_tab)
srot60_results = Frame(results_tab)
dRL3_results = Frame(results_tab)
x_val_fr = Frame(results_tab)

#Tittles for frames
label_res0= Label(srot0_results,text="RL3 for SROT 0")
label_res0.pack(side=TOP)
label_res60= Label(srot60_results,text="RL3 for SROT 60")
label_res60.pack(side=TOP)
label_dRL3= Label(dRL3_results,text="delta RL3")
label_dRL3.pack(side=TOP)


#Scrollbars
scroll_res0 = Scrollbar(srot0_results, orient = VERTICAL)
scroll_res60 = Scrollbar(srot60_results, orient = VERTICAL)
scroll_resdRL3 = Scrollbar(dRL3_results, orient = VERTICAL)


#Listboxes (tree)
columns_RL3 = ('number of meaurement','RL factor')
columns_dRL3 = ('number of meaurement srot 0', 'number of meaurement srot 60', 'dRL factor')


res0_tree = ttk.Treeview(srot0_results, columns=columns_RL3, show='headings') #results of RL3 for srot 0
res60_tree = ttk.Treeview(srot60_results, columns=columns_RL3, show='headings') #results of RL3 for srot 60
resdRL3_tree = ttk.Treeview(dRL3_results, columns=columns_dRL3, show='headings') #results of delta RL3

#Headings for trees (not containing type of RL)
res0_tree.heading('number of meaurement', text='Number')
res60_tree.heading('number of meaurement', text='Number')
resdRL3_tree.heading('number of meaurement srot 0', text='Number (srot0)')
resdRL3_tree.heading('number of meaurement srot 60', text='Number (srot60)')


#Buttons
plt_res_btn = Button(results_tab, text = 'Plot results' , command = plot_results)
save_file_btn = Button(results_tab, text = 'Save to file' , command = lambda : save_to_file())

#Checkbuttons
if_specify = IntVar()
check_x_val = Checkbutton(results_tab, text = "Do you want to specify x values for the plot?",
                          variable=if_specify, command = specify_x)

#Entries
x_tittle_ent = Entry(x_val_fr)
x_val_ent = Entry(x_val_fr)

#Tittles for entries
label_x_tittle = Label(x_val_fr, text="X axis tittle")
label_x_tittle.grid(row = 0, column = 0)
label_x_val = Label(x_val_fr, text="X values (separate with white space)")
label_x_val.grid(row = 1, column = 0)

#Packing widgets
#Frames
srot0_results.grid(row = 0, column = 0, columnspan = 2, padx = 10, pady = 10)
srot60_results.grid(row = 0, column = 2, columnspan = 2, padx = 10, pady = 10)
dRL3_results.grid(row = 1, column = 0, columnspan = 3, rowspan = 4, padx = 10, pady = 10)


#Listboxesx
res0_tree.pack(side = LEFT, fill = BOTH)
res60_tree.pack(side = LEFT, fill = BOTH)
resdRL3_tree.pack(side = LEFT, fill = BOTH)

#Scrollbars
scroll_res0.pack(side = RIGHT, fill = Y)
scroll_res0.config(command = ms_list.yview)
scroll_res60.pack(side = RIGHT, fill = Y)
scroll_res60.config(command = ms_list.yview)
scroll_resdRL3.pack(side = RIGHT, fill = Y)
scroll_resdRL3.config(command = ms_list.yview)

#Buttons
plt_res_btn.grid(row = 3, column = 3)
save_file_btn.grid(row = 4, column = 3)

#Checkbuttons
check_x_val.grid(row = 1, column = 3)

#Entries
x_tittle_ent.grid(row = 0, column = 1)
x_val_ent.grid(row = 1, column = 1)



root.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "D:\Programy\anaconda\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "C:\Users\Maciek\AppData\Local\Temp\ipykernel_6624\3951112773.py", line 10, in select_file
    data_set = open(file_path, 'r')
FileNotFoundError: [Errno 2] No such file or directory: ''


(35, 777.2510443023469)
